In [47]:
import pandas as pd
import numpy as np
import re

_MESES = {
    "enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06",
    "julio":"07","agosto":"08","septiembre":"09","setiembre":"09","octubre":"10",
    "noviembre":"11","diciembre":"12"
}
_PAT_ES = re.compile(r"^\s*(\d{1,2})\s+de\s+([A-Za-záéíóúñÑ]+)\s+de\s+(\d{4})", re.IGNORECASE)

def parse_fecha_es(x):
    if pd.isna(x): 
        return pd.NaT
    s = str(x).strip()
    m = _PAT_ES.match(s)
    if m:
        d, mes, y = m.groups()
        mm = _MESES.get(mes.lower())
        if mm:
            return pd.to_datetime(f"{y}-{mm}-{int(d):02d}", errors="coerce")
    return pd.to_datetime(s, errors="coerce", dayfirst=True)

def clean_balance_data(df_balance: pd.DataFrame) -> pd.DataFrame:
    df = df_balance.copy()

    date_col = "balance_date" if "balance_date" in df.columns else "operation_date"
    df["operation_date"] = df[date_col].apply(parse_fecha_es)

    required = {"contract","value_pos_mdo","operation_date"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"Faltan columnas en balances: {missing}")

    df = df.dropna(subset=["operation_date", "contract", "value_pos_mdo"])
    df["contract"] = df["contract"].astype(str)

    df_clean = (
        df.groupby(["contract","operation_date"], as_index=False)["value_pos_mdo"]
          .sum()
          .rename(columns={"value_pos_mdo":"portfolio_value"})
          .sort_values(["contract","operation_date"])
          .reset_index(drop=True)
    )
    return df_clean

_DEPOS_PAT = re.compile(r"DEPOSITO", re.IGNORECASE)
_RETIRO_PAT = re.compile(r"RETIRO", re.IGNORECASE)

def clean_movements_data(df_mov: pd.DataFrame) -> pd.DataFrame:
    df = df_mov.copy()

    if "operation_date" in df.columns:
        date_col = "operation_date"
    elif "movement_date" in df.columns:
        date_col = "movement_date"
    else:
        date_col = "settlement_date"

    df["operation_date"] = df[date_col].apply(parse_fecha_es)

    required = {"contract", "movement_import", "description", "operation_date"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"Faltan columnas en movements: {missing}")

    df = df.dropna(subset=["operation_date", "contract", "movement_import", "description"])
    df["contract"] = df["contract"].astype(str)

    mask = df["description"].str.contains("DEPOSITO", case=False, na=False) | \
           df["description"].str.contains("RETIRO",   case=False, na=False)
    df = df.loc[mask, ["contract","description","movement_import","operation_date"]].copy()

    def signed_amount(row):
        amt = abs(float(row["movement_import"]))
        if _DEPOS_PAT.search(str(row["description"])):
            return -amt   
        if _RETIRO_PAT.search(str(row["description"])):
            return +amt   
        return 0.0

    df["cash_flow"] = df.apply(signed_amount, axis=1)
    df = df.sort_values(["contract","operation_date"]).reset_index(drop=True)
    return df[["contract","operation_date","description","cash_flow"]]

def _xirr_fallback(dates, cashflows, guess=0.1, max_iter=200, tol=1e-7):
    dates = pd.to_datetime(dates)
    t0 = dates.min()
    days = (dates - t0).days.astype(float)

    def f(r):
        return np.sum([cf / (1 + r)**(d/365.0) for cf, d in zip(cashflows, days)])
    def fp(r):
        return np.sum([-(d/365.0) * cf / (1 + r)**(d/365.0 + 1) for cf, d in zip(cashflows, days)])

    r = guess
    for _ in range(max_iter):
        fr = f(r)
        if abs(fr) < tol:
            return r
        dfr = fp(r)
        if dfr == 0:
            r += 1e-4
            continue
        r_next = r - fr/dfr
        if r_next <= -0.999999:
            r_next = (r - 0.5) if r > -0.5 else r + 0.5
        if abs(r_next - r) < tol:
            return r_next
        r = r_next
    return r

def xirr_auto(dates, cashflows):
    try:
        from pyxirr import xirr as _xirr
        return float(_xirr(dates, cashflows))
    except Exception:
        return float(_xirr_fallback(dates, cashflows))

def MWRR(balance_df: pd.DataFrame, movements_df: pd.DataFrame, contrato: str) -> float:
    contrato = str(contrato)
    bal = balance_df[balance_df["contract"] == contrato].copy()
    mov = movements_df[movements_df["contract"] == contrato].copy()

    if bal.empty:
        raise ValueError(f"Sin balances para contrato {contrato}")
    
    bal = bal.sort_values("operation_date")
    last_date = bal.iloc[-1]["operation_date"]
    last_value = float(bal.iloc[-1]["portfolio_value"])

    fechas = mov["operation_date"].tolist() + [last_date]
    flujos = mov["cash_flow"].tolist() + [last_value]

    return xirr_auto(fechas, flujos)

if __name__ == "__main__":
    file_path = "data_actividad.xlsx" 

    # Carga
    df_movements = pd.read_excel(file_path, sheet_name="movements")
    df_balance   = pd.read_excel(file_path, sheet_name="balances")  # OJO: en tu archivo es 'balances'

    # Limpieza
    clean_bal = clean_balance_data(df_balance)
    clean_mov = clean_movements_data(df_movements)

    # Cálculo
    contratos = ["20486403", "12861603", "AHA84901"]
    for c in contratos:
        r = MWRR(clean_bal, clean_mov, c)
        print(f"Rendimiento cliente {c}: {r*100:.2f}%")


Rendimiento cliente 20486403: 8432.85%
Rendimiento cliente 12861603: 11.69%
Rendimiento cliente AHA84901: 11.05%
